# Learn-SQL

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master('local').appName('learn-sql').getOrCreate()

23/05/07 00:03:16 WARN Utils: Your hostname, Keemyoui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.35.79 instead (on interface en0)
23/05/07 00:03:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/07 00:03:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/07 00:03:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/07 00:03:18 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
stocks = [
    ("Google", "GOOGL", "USA", 2984, "USD"),
    ("Netflix", "NFLX", "USA", 645, "USD"),
    ("Amazon", "AMZN", "USA", 3518, "USD"),
    ("Tesla", "TSLA", "USA", 1222, "USD"),
    ("Tencent", "0700", "Hong Kong", 483, "HKD"),
    ("Toyota", "7203", "Japan", 2006, "JPY"),
    ("Samsung", "005930", "Korea", 70600, "KRW"),
    ("Kakao", "035720", "Korea", 125000, "KRW"),
]

In [5]:
stockSchema = ['name', 'ticker', 'country', 'price', 'currency']

In [6]:
# data type을 스스로 infer
df = spark.createDataFrame(data = stocks, schema = stockSchema)
df.dtypes

[('name', 'string'),
 ('ticker', 'string'),
 ('country', 'string'),
 ('price', 'bigint'),
 ('currency', 'string')]

In [7]:
df.show()

+-------+------+---------+------+--------+
|   name|ticker|  country| price|currency|
+-------+------+---------+------+--------+
| Google| GOOGL|      USA|  2984|     USD|
|Netflix|  NFLX|      USA|   645|     USD|
| Amazon|  AMZN|      USA|  3518|     USD|
|  Tesla|  TSLA|      USA|  1222|     USD|
|Tencent|  0700|Hong Kong|   483|     HKD|
| Toyota|  7203|    Japan|  2006|     JPY|
|Samsung|005930|    Korea| 70600|     KRW|
|  Kakao|035720|    Korea|125000|     KRW|
+-------+------+---------+------+--------+



In [8]:
# 스파크 SQL을 쓸건데, 그럼 temporary view에 등록해야함
df.createOrReplaceTempView("stocks")

In [10]:
# 각 컬럼을 갖고 오게 된다.
spark.sql('select name from stocks').show()

+-------+
|   name|
+-------+
| Google|
|Netflix|
| Amazon|
|  Tesla|
|Tencent|
| Toyota|
|Samsung|
|  Kakao|
+-------+



In [12]:
spark.sql("SELECT name, price from stocks").show()

+-------+------+
|   name| price|
+-------+------+
| Google|  2984|
|Netflix|   645|
| Amazon|  3518|
|  Tesla|  1222|
|Tencent|   483|
| Toyota|  2006|
|Samsung| 70600|
|  Kakao|125000|
+-------+------+



In [18]:
# 조건문을 갖고와보자 (1)
spark.sql("SELECT name, price from stocks where country = 'Korea'").show()

+-------+------+
|   name| price|
+-------+------+
|Samsung| 70600|
|  Kakao|125000|
+-------+------+



In [19]:
# 조건문을 갖고와보자 (2)
spark.sql("SELECT name, price from stocks where price > 2000").show()

+-------+------+
|   name| price|
+-------+------+
| Google|  2984|
| Amazon|  3518|
| Toyota|  2006|
|Samsung| 70600|
|  Kakao|125000|
+-------+------+



In [21]:
# 조건문을 갖고와보자 (3)
spark.sql("SELECT name, price from stocks where price > 2000 and country = 'USA'").show()

+------+-----+
|  name|price|
+------+-----+
|Google| 2984|
|Amazon| 3518|
+------+-----+



In [23]:
spark.sql('select name, price from stocks where country like "U%"').show()

+-------+-----+
|   name|price|
+-------+-----+
| Google| 2984|
|Netflix|  645|
| Amazon| 3518|
|  Tesla| 1222|
+-------+-----+



In [24]:
spark.sql('select name, price from stocks where country like "U%" and name not like "%e%"').show()

+------+-----+
|  name|price|
+------+-----+
|Amazon| 3518|
+------+-----+



In [25]:
spark.sql('select name, price from stocks where price between 1000 and 10000').show()

+------+-----+
|  name|price|
+------+-----+
|Google| 2984|
|Amazon| 3518|
| Tesla| 1222|
|Toyota| 2006|
+------+-----+



In [27]:
spark.sql('select name, price from stocks where country="USA"').show()

+-------+-----+
|   name|price|
+-------+-----+
| Google| 2984|
|Netflix|  645|
| Amazon| 3518|
|  Tesla| 1222|
+-------+-----+



In [28]:
spark.sql('select name, price, currency from stocks \
where 1=1 \
    AND currency = "USD" \
    AND price < (select price from stocks where name = "Tesla")').show()

+-------+-----+--------+
|   name|price|currency|
+-------+-----+--------+
|Netflix|  645|     USD|
+-------+-----+--------+



In [29]:
spark.sql('select name, price from stocks order by price asc').show()

+-------+------+
|   name| price|
+-------+------+
|Tencent|   483|
|Netflix|   645|
|  Tesla|  1222|
| Toyota|  2006|
| Google|  2984|
| Amazon|  3518|
|Samsung| 70600|
|  Kakao|125000|
+-------+------+



In [31]:
spark.sql('select name, price from stocks order by length(name)').show()

+-------+------+
|   name| price|
+-------+------+
|  Tesla|  1222|
|  Kakao|125000|
| Amazon|  3518|
| Toyota|  2006|
| Google|  2984|
|Netflix|   645|
|Samsung| 70600|
|Tencent|   483|
+-------+------+



In [32]:
spark.sql("select sum(price) from stocks where country = 'Korea'").show()

+----------+
|sum(price)|
+----------+
|    195600|
+----------+



In [33]:
spark.sql("select count(price) from stocks where country in ('Korea', 'USA')").show()

+------------+
|count(price)|
+------------+
|           6|
+------------+



In [34]:
earnings = [
    ('Google', 27.99, 'USD'),
    ('Netflix', 2.56, 'USD'),
    ('Amazon', 6.12, 'USD'),
    ('Tesla', 1.86, 'USD'),
    ('Tencent', 11.01, 'HKD'),
    ('Toyota', 224.82, 'JPY'),
    ('Samsung', 1780., 'KRW'),
    ('Kakao', 705., 'KRW'),
]

In [35]:
from pyspark.sql.types import StringType, FloatType, StructType, StructField

In [37]:
earningsSchema = StructType([
    StructField('name', StringType(), True),
    StructField('eps', FloatType(), True),
    StructField('currency', StringType(), True),        
])

In [38]:
earningsDF = spark.createDataFrame(data=earnings, schema = earningsSchema) 

EarningsDF도 sql에서 사용하기 위해 temporary View에 등록을 해줘야함

In [39]:
earningsDF.dtypes

[('name', 'string'), ('eps', 'float'), ('currency', 'string')]

In [40]:
earningsDF.createOrReplaceTempView('earnings')

In [42]:
earningsDF.select("*").show()

+-------+------+--------+
|   name|   eps|currency|
+-------+------+--------+
| Google| 27.99|     USD|
|Netflix|  2.56|     USD|
| Amazon|  6.12|     USD|
|  Tesla|  1.86|     USD|
|Tencent| 11.01|     HKD|
| Toyota|224.82|     JPY|
|Samsung|1780.0|     KRW|
|  Kakao| 705.0|     KRW|
+-------+------+--------+



In [44]:
spark.sql("select * from stocks join earnings on stocks.name = earnings.name").show()

+-------+------+---------+------+--------+-------+------+--------+
|   name|ticker|  country| price|currency|   name|   eps|currency|
+-------+------+---------+------+--------+-------+------+--------+
| Amazon|  AMZN|      USA|  3518|     USD| Amazon|  6.12|     USD|
| Google| GOOGL|      USA|  2984|     USD| Google| 27.99|     USD|
|  Kakao|035720|    Korea|125000|     KRW|  Kakao| 705.0|     KRW|
|Netflix|  NFLX|      USA|   645|     USD|Netflix|  2.56|     USD|
|Samsung|005930|    Korea| 70600|     KRW|Samsung|1780.0|     KRW|
|Tencent|  0700|Hong Kong|   483|     HKD|Tencent| 11.01|     HKD|
|  Tesla|  TSLA|      USA|  1222|     USD|  Tesla|  1.86|     USD|
| Toyota|  7203|    Japan|  2006|     JPY| Toyota|224.82|     JPY|
+-------+------+---------+------+--------+-------+------+--------+



In [48]:
# PER: Price / EPS
spark.sql('select stocks.name, round(stocks.price/earnings.eps,1) as PER from stocks join earnings on stocks.name = earnings.name').show()

+-------+-----+
|   name|  PER|
+-------+-----+
| Amazon|574.8|
| Google|106.6|
|  Kakao|177.3|
|Netflix|252.0|
|Samsung| 39.7|
|Tencent| 43.9|
|  Tesla|657.0|
| Toyota|  8.9|
+-------+-----+

